In [1]:
import scenicplus
import muon as mu
import pycisTopic
import pandas as pd
from pycisTopic.plotting.qc_plot import plot_sample_stats, plot_barcode_stats
import matplotlib.pyplot as plt
import os
import pickle
from pycisTopic.qc import get_barcodes_passing_qc_for_sample
from pycisTopic.pseudobulk_peak_calling import export_pseudobulk
from pycisTopic.pseudobulk_peak_calling import peak_calling
from pycisTopic.iterative_peak_calling import get_consensus_peaks
from pycisTopic.iterative_peak_calling import get_consensus_peaks
from pycisTopic.lda_models import evaluate_models




out_dir = "chen_outs"
os.makedirs(out_dir, exist_ok = True)
os.makedirs(os.path.join(out_dir, "consensus_peak_calling"), exist_ok = True)
os.makedirs(os.path.join(out_dir, "consensus_peak_calling/pseudobulk_bed_files"), exist_ok = True)
os.makedirs(os.path.join(out_dir, "consensus_peak_calling/pseudobulk_bw_files"), exist_ok = True)

/data/adrienne/phd/biohackathon_2025/scenicplus/src/scenicplus/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound
/home/adrienne/mambaforge/envs/scenicplus_release/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-05 21:26:00,523	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
print(pycisTopic.__version__)
print(scenicplus.__version__)

2.0a0
1.0a2


In [3]:
res_folder = 'analysis_chen/results/chen_2024/'
samples = os.listdir(res_folder)

In [31]:
mdata = mu.read_h5mu('analysis_chen/integration_test_3.h5mu')

/home/adrienne/mambaforge/envs/scenicplus_release/lib/python3.11/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/adrienne/mambaforge/envs/scenicplus_release/lib/python3.11/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(


In [32]:
cell_data = mdata.obs

In [6]:
cistopic_list = []
for sample in samples:
    with open(os.path.join(out_dir, f"{sample}_cistopic_obj.pkl"), "rb") as f:
        cistopic_list.append(pickle.load(f))

In [7]:
cistopic_list[0].merge(cistopic_list[1:])

2025-09-05 21:26:33,138 cisTopic     INFO     cisTopic object 1 merged
2025-09-05 21:26:45,647 cisTopic     INFO     cisTopic object 2 merged
2025-09-05 21:27:00,455 cisTopic     INFO     cisTopic object 3 merged
2025-09-05 21:27:16,736 cisTopic     INFO     cisTopic object 4 merged
2025-09-05 21:27:35,618 cisTopic     INFO     cisTopic object 5 merged
2025-09-05 21:27:57,456 cisTopic     INFO     cisTopic object 6 merged
2025-09-05 21:28:25,696 cisTopic     INFO     cisTopic object 7 merged


In [8]:
cistopic_merged = cistopic_list[0]

In [9]:
del cistopic_list

In [10]:
cistopic_merged.cell_names

['CGTTAGGTCTCCTCTT-1-chen_10wk_central___chen_10wk_central',
 'GGCAAGCCAGTAGGAT-1-chen_10wk_central___chen_10wk_central',
 'GGGCTAACAAATACCT-1-chen_10wk_central___chen_10wk_central',
 'ACGGTACGTCAATAGT-1-chen_10wk_central___chen_10wk_central',
 'TCAAACCAGTCACTAA-1-chen_10wk_central___chen_10wk_central',
 'AGGCGGATCTCACTCA-1-chen_10wk_central___chen_10wk_central',
 'TTGCATTTCAAATCGC-1-chen_10wk_central___chen_10wk_central',
 'ACCTACCTCCGGTTAG-1-chen_10wk_central___chen_10wk_central',
 'ATTAGCTCAAGTCGCT-1-chen_10wk_central___chen_10wk_central',
 'GTATTCAAGTTCCTCA-1-chen_10wk_central___chen_10wk_central',
 'TAAGGCTTCCTAAGTA-1-chen_10wk_central___chen_10wk_central',
 'CAGGATGCACTTGTTC-1-chen_10wk_central___chen_10wk_central',
 'ACCCGTAAGCGTGCGT-1-chen_10wk_central___chen_10wk_central',
 'GGCTAGTGTGCATTAG-1-chen_10wk_central___chen_10wk_central',
 'GCACCTAAGCTCCTAC-1-chen_10wk_central___chen_10wk_central',
 'ATTAACCCAGAACCGA-1-chen_10wk_central___chen_10wk_central',
 'GCCTTAGAGGACTTAC-1-che

In [18]:
cell_data.iloc[0]

rna:sample                chen_10wk_central
rna:age                                10wk
rna:majorclass                         PRPC
rna:subclass                           PRPC
rna:n_genes_by_counts                  2913
rna:total_counts                     5635.0
rna:total_counts_mt                    16.0
rna:pct_counts_mt                   0.28394
rna:scDblFinder_score              0.000021
rna:scDblFinder_class               singlet
rna:_scvi_batch                           0
rna:_scvi_labels                          0
rna:leiden                               18
atac:n_fragment                        2559
atac:frac_dup                      0.435473
atac:frac_mito                          0.0
atac:tsse                         12.558575
atac:sample               chen_10wk_central
atac:age                               10wk
atac:majorclass                        PRPC
atac:subclass                          PRPC
atac:scDblFinder_score             0.000714
atac:scDblFinder_class          

In [33]:
cell_data.index = [i.replace('1_', '1-') + '___' + cell_data.loc[i]['atac:sample'] for i in cell_data.index]

In [35]:
cell_data

,rna:sample,rna:age,rna:majorclass,rna:subclass,rna:n_genes_by_counts,rna:total_counts,rna:total_counts_mt,rna:pct_counts_mt,rna:scDblFinder_score,rna:scDblFinder_class,...,atac:scDblFinder_score,atac:scDblFinder_class,atac:_scvi_batch,atac:_scvi_labels,atac:leiden,sample,_indices,_scvi_batch,_scvi_labels,leiden
AAACAGCCAAACTGTT-1-chen_10wk_central___chen_10wk_central,chen_10wk_central,10wk,PRPC,PRPC,2913,5635.0,16.0,0.283940,0.000021,singlet,...,0.000714,singlet,0,0,11,chen_10wk_central,0,0,0,6
AAACAGCCAATGAGGT-1-chen_10wk_central___chen_10wk_central,chen_10wk_central,10wk,RGC,RGC Precursor,3241,7922.0,109.0,1.375915,0.000021,singlet,...,0.140442,singlet,0,0,7,chen_10wk_central,1,0,0,8
AAACAGCCAGCTTAGC-1-chen_10wk_central___chen_10wk_central,chen_10wk_central,10wk,PRPC,PRPC,4068,9670.0,11.0,0.113754,0.000028,singlet,...,0.002203,singlet,0,0,1,chen_10wk_central,2,0,0,6
AAACATGCAGCTTACA-1-chen_10wk_central___chen_10wk_central,chen_10wk_central,10wk,RGC,OFF_MGC,4060,11122.0,18.0,0.161841,0.000021,singlet,...,0.002927,singlet,0,0,16,chen_10wk_central,3,0,0,15
AAACATGCATGTTGTG-1-chen_10wk_central___chen_10wk_central,chen_10wk_central,10wk,RGC,RGC Precursor,4331,12910.0,84.0,0.650658,0.000021,singlet,...,0.024031,singlet,0,0,16,chen_10wk_central,4,0,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTGTTCTAGCGTG-1-chen_20wk2d_central___chen_20wk2d_central,chen_20wk2d_central,20wk2d,AC,GABAergic,3032,7746.0,46.0,0.593855,0.000062,singlet,...,0.000342,singlet,0,0,5,chen_20wk2d_central,38505,6,0,16
TTTGTGTTCTCAATAG-1-chen_20wk2d_central___chen_20wk2d_central,chen_20wk2d_central,20wk2d,Rod,Rod Precursor,1749,3186.0,4.0,0.125549,0.000021,singlet,...,0.000413,singlet,0,0,14,chen_20wk2d_central,38506,6,0,4
TTTGTTGGTACAAAGA-1-chen_20wk2d_central___chen_20wk2d_central,chen_20wk2d_central,NaN,NaN,NaN,2448,5973.0,6.0,0.100452,0.000028,singlet,...,0.004614,singlet,0,0,15,chen_20wk2d_central,38507,6,0,13
TTTGTTGGTCAAGTAT-1-chen_20wk2d_central___chen_20wk2d_central,chen_20wk2d_central,20wk2d,Rod,Rod,3736,10123.0,16.0,0.158056,0.000084,singlet,...,0.012636,singlet,0,0,14,chen_20wk2d_central,38508,6,0,4


In [30]:
cistopic_merged.cell_names

['CGTTAGGTCTCCTCTT-1-chen_10wk_central___chen_10wk_central',
 'GGCAAGCCAGTAGGAT-1-chen_10wk_central___chen_10wk_central',
 'GGGCTAACAAATACCT-1-chen_10wk_central___chen_10wk_central',
 'ACGGTACGTCAATAGT-1-chen_10wk_central___chen_10wk_central',
 'TCAAACCAGTCACTAA-1-chen_10wk_central___chen_10wk_central',
 'AGGCGGATCTCACTCA-1-chen_10wk_central___chen_10wk_central',
 'TTGCATTTCAAATCGC-1-chen_10wk_central___chen_10wk_central',
 'ACCTACCTCCGGTTAG-1-chen_10wk_central___chen_10wk_central',
 'ATTAGCTCAAGTCGCT-1-chen_10wk_central___chen_10wk_central',
 'GTATTCAAGTTCCTCA-1-chen_10wk_central___chen_10wk_central',
 'TAAGGCTTCCTAAGTA-1-chen_10wk_central___chen_10wk_central',
 'CAGGATGCACTTGTTC-1-chen_10wk_central___chen_10wk_central',
 'ACCCGTAAGCGTGCGT-1-chen_10wk_central___chen_10wk_central',
 'GGCTAGTGTGCATTAG-1-chen_10wk_central___chen_10wk_central',
 'GCACCTAAGCTCCTAC-1-chen_10wk_central___chen_10wk_central',
 'ATTAACCCAGAACCGA-1-chen_10wk_central___chen_10wk_central',
 'GCCTTAGAGGACTTAC-1-che

In [39]:
cell_names_overlap = set(cistopic_merged.cell_names).intersection(set(cell_data.index)) 

In [41]:
cell_data = cell_data.loc[cell_names_overlap]

/tmp/ipykernel_56980/2724406967.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  cell_data = cell_data.loc[cell_names_overlap]


In [42]:
cell_data

,rna:sample,rna:age,rna:majorclass,rna:subclass,rna:n_genes_by_counts,rna:total_counts,rna:total_counts_mt,rna:pct_counts_mt,rna:scDblFinder_score,rna:scDblFinder_class,...,atac:scDblFinder_score,atac:scDblFinder_class,atac:_scvi_batch,atac:_scvi_labels,atac:leiden,sample,_indices,_scvi_batch,_scvi_labels,leiden
ACCAGCTCATTAAAGG-1-chen_10wk_central___chen_10wk_central,chen_10wk_central,10wk,RGC,RGC Precursor,3731,9891.0,229.0,2.315236,0.000021,singlet,...,0.005242,singlet,0,0,7,chen_10wk_central,276,0,0,8
TAACAAGCAGCTCAAC-1-chen_20wk2d_peripheral___chen_20wk2d_peripheral,chen_20wk2d_peripheral,20wk2d,Rod,Rod Precursor,1515,3323.0,2.0,0.060187,0.000021,singlet,...,0.000346,singlet,0,0,0,chen_20wk2d_peripheral,11358,7,0,11
GGATGTAAGCAAGACA-1-chen_10wk_central___chen_10wk_central,chen_10wk_central,10wk,RGC,RGC Precursor,3864,9591.0,109.0,1.136482,0.000021,singlet,...,0.216000,singlet,0,0,7,chen_10wk_central,2166,0,0,8
AGCCAGTGTTGTTGTC-1-chen_10wk_central___chen_10wk_central,chen_10wk_central,10wk,NRPC,NRPC,1934,3429.0,4.0,0.116652,0.000021,singlet,...,0.267238,singlet,0,0,7,chen_10wk_central,487,0,0,8
TAAGCTGGTGTCACGG-1-chen_20wk2d_peripheral___chen_20wk2d_peripheral,chen_20wk2d_peripheral,20wk2d,Rod,Rod Precursor,3162,7566.0,27.0,0.356860,0.000023,singlet,...,0.000579,singlet,0,0,14,chen_20wk2d_peripheral,11391,7,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GGACGGATCCTAGTTT-1-chen_10wk_central___chen_10wk_central,chen_10wk_central,10wk,RGC,RGC Precursor,3940,11215.0,107.0,0.954079,0.000035,singlet,...,0.033980,singlet,0,0,7,chen_10wk_central,2134,0,0,8
ACCGCAATCGCGACAC-1-chen_16wk4d_peripheral___chen_16wk4d_peripheral,chen_16wk4d_peripheral,16wk4d,PRPC,PRPC,2400,4776.0,11.0,0.230318,0.000021,singlet,...,0.004437,singlet,0,0,19,chen_16wk4d_peripheral,18661,5,0,3
CTCAATAAGCACAGAA-1-chen_20wk2d_central___chen_20wk2d_central,chen_20wk2d_central,NaN,NaN,NaN,2914,9146.0,46.0,0.502952,0.000084,singlet,...,0.000361,singlet,0,0,12,chen_20wk2d_central,34369,6,0,13
TTTAAGGTCGTCAAGT-1-chen_12wk3d_peripheral___chen_12wk3d_peripheral,chen_12wk3d_peripheral,12wk3d,PRPC,PRPC,2029,3353.0,14.0,0.417537,0.000021,singlet,...,0.002935,singlet,0,0,18,chen_12wk3d_peripheral,31251,3,0,3


In [43]:
cell_data.head()
cistopic_merged.add_cell_data(cell_data, split_pattern='-')


Columns ['atac:subclass', 'atac:scDblFinder_class', 'rna:scDblFinder_score', 'rna:majorclass', 'rna:total_counts', 'rna:total_counts_mt', 'rna:pct_counts_mt', 'atac:_scvi_batch', 'atac:_scvi_labels', 'atac:frac_mito', 'rna:age', 'rna:n_genes_by_counts', 'sample', 'rna:_scvi_batch', 'atac:majorclass', 'rna:leiden', 'rna:_scvi_labels', 'atac:n_fragment', 'atac:frac_dup', 'atac:age', 'atac:tsse', 'atac:sample', 'rna:scDblFinder_class', 'rna:subclass', 'atac:leiden', '_scvi_labels', '_indices', 'rna:sample', 'atac:scDblFinder_score', 'leiden', '_scvi_batch'] will be overwritten


!wget https://github.com/mimno/Mallet/releases/download/v202108/Mallet-202108-bin.tar.gz
!tar -xf Mallet-202108-bin.tar.gz

In [44]:
pickle.dump(cistopic_merged, open(os.path.join(out_dir, "chen_cistopic_obj_merged_annotated_1.pkl"), "wb"))